                         NER for detecting Drugs names

Downloading Packages

In [18]:
import random
import pandas as pd
import spacy
import re
from spacy.training import Example
import spacy.cli

Making the spacy module to use gpu only while training.

In [19]:
spacy.prefer_gpu

<function thinc.util.prefer_gpu(gpu_id: int = 0) -> bool>

loading english pipeline

In [20]:
nlp = spacy.load("en_core_web_sm")

loading Dataset

In [21]:
df = pd.read_csv("drug_review_dataset_with_sentiment.csv")

/tmp/ipykernel_15551/133009624.py:1: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("drug_review_dataset_with_sentiment.csv")


Tokenizing 

In [22]:
def process_review(review):
    processed_token = []
    for token in review.split():
        token = ''.join(e.lower() for e in token if e.isalnum())
        processed_token.append(token)
    return ' '.join(processed_token)

Labeling

In [23]:
all_drugs = df['drugName'].unique().tolist()
all_drugs = [x.lower() for x in all_drugs]
all_drugs
df['review']
LABEL = 'DRUG'

preparing data for training

In [24]:
count = 0
TRAIN_DATA = []
for _, item in df.iterrows():
    ent_dict = {}
    if count < 1000:
        review = process_review(item['review'])
        #Locate drugs and their positions once and add to the visited items.
        visited_items = []
        entities = []
        for token in review.split():
            if token in all_drugs:
                for i in re.finditer(token, review):
                    if token not in visited_items:
                        entity = (i.span()[0], i.span()[1], 'DRUG')
                        visited_items.append(token)
                        entities.append(entity)
        if len(entities) > 0:
            ent_dict['entities'] = entities
            train_item = (review, ent_dict)
            TRAIN_DATA.append(train_item)
            count+=1

Loading the spacy models for labeling

In [25]:
nlp = spacy.load('en_core_web_sm')  # load existing spaCy model
ner = nlp.get_pipe('ner')
ner.add_label(LABEL)


1

Optimization of model

In [26]:
optimizer = nlp.create_optimizer()

Removing extra pipes and minimizing the loss during training

In [27]:
other_pipes = [pipe for pipe in nlp.pipe_names if pipe != "ner"]
with nlp.disable_pipes(*other_pipes):  # only train NER
    for itn in range(20):
        random.shuffle(TRAIN_DATA)
        losses = {}
        for text, annotations in TRAIN_DATA:
            doc = nlp.make_doc(text)
            example = Example.from_dict(doc, annotations)
            nlp.update([example], drop=0.35, sgd=optimizer, losses=losses)
        print(losses)

/home/utkarsh/.local/lib/python3.10/site-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "awful medicine the worst the side effects outweigh..." with entities "[(178, 183, 'DRUG'), (342, 350, 'DRUG')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
/home/utkarsh/.local/lib/python3.10/site-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "many people are unreasonably scared of klonopin039..." with entities "[(39, 47, 'DRUG')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
/home/utkarsh/.local/lib/python3.10/site-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the tex

{'ner': 1885.7641941293439}
{'ner': 1236.587471304004}
{'ner': 1057.6603234226434}
{'ner': 818.7887025848779}
{'ner': 704.551107814492}
{'ner': 646.4032097017794}
{'ner': 596.8705891367757}
{'ner': 542.2245839852097}
{'ner': 473.41817450651945}
{'ner': 528.9840122235906}
{'ner': 411.8451903461538}
{'ner': 344.80751057085075}
{'ner': 389.182265255289}
{'ner': 324.1690997652709}
{'ner': 281.55238451814444}
{'ner': 232.98619763855265}
{'ner': 239.27875182711858}
{'ner': 297.3544389158859}
{'ner': 214.7401241651389}
{'ner': 159.4508869560689}


Saving traind model

In [28]:
nlp.to_disk('NER_VoltaML_new_utkarsh.spacy')

Testing by loading saved ner model

In [29]:
def listToString(s): 
    str1 = " " 
    return (str1.join(s))
nlp = spacy.load('NER_VoltaML_new_utkarsh.spacy')

providing sample text

In [30]:
test_text = "Caja con 30 tabletas ibuprofen quetiapina 25 mg Tabletas AstraZeneca2"

Printing Results


In [31]:
doc = nlp(test_text)
print("Entities in '%s'" % test_text)
for ent in doc.ents:
    print(ent.label_, " -- ", ent.text)


Entities in 'Caja con 30 tabletas ibuprofen quetiapina 25 mg Tabletas AstraZeneca2'
